# Process tomography

In [ ]:
import numpy as np
from pyquil.api import get_qc
from pyquil import Program
from pyquil.gates import *

from forest_qcvv.tomography import *
from forest_qcvv.superop_conversion import choi2pauli_liouville, kraus2choi


## Set the shots, the qubits of interest, the quantum device

In [ ]:
qubits = [0]
qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=True)

# Create a Process

In [ ]:
process = Program(RX(np.pi, qubits[0]))

In [ ]:
exp_desc = generate_process_tomography_experiment(process)

In [ ]:
print(exp_desc.program)

In [ ]:
[str(exp_desc.out_ops[idx]) for idx in list(range(0,(2**len(qubits))**2 - 1))]

# Acquire data

In [ ]:
exp_data = acquire_tomography_data(exp_desc, qvm)

In [ ]:
exp_data

# PGDB Estimate

In [ ]:
estimate = pgdb_process_estimate(exp_data)

In [ ]:
print(np.real(np.round(estimate.estimate.process_choi_est, 2)))

In [ ]:
ideal = np.array([[0.,0.,0.,0.],[0.,1.,1.,0.],[0.,1.,1.,0.],[0.,0.,0.,0.]])
print(ideal)

# Plot Pauli Transfer Matrix of Estimate

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ptm = choi2pauli_liouville(estimate.estimate.process_choi_est)
labels = [op.pauli_string for op in estimate.out_ops]
plot_pauli_transfer_matrix(ptm, ax, title='Estimate')

In [ ]:
fig, ax = plt.subplots()
ideal_ptm = choi2pauli_liouville(ideal)
labels = [op.pauli_string for op in estimate.out_ops]
plot_pauli_transfer_matrix(ideal_ptm, ax, title='Ideal')

# Two qubit example (slow)

In [ ]:
from scipy.linalg import expm
from numpy import pi

process = Program(CNOT(1,0))

exp_desc = generate_process_tomography_experiment(process)
exp_data = acquire_tomography_data(exp_desc, qvm)
estimate = pgdb_process_estimate(exp_data)
for row in estimate.estimate.process_choi_est:
    print([abs(np.real(np.round(r,1))) for r in row])

In [ ]:
ideal = kraus2choi(np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]]))
print(ideal)

In [ ]:
fig, ax = plt.subplots()
ptm = choi2pauli_liouville(estimate.estimate.process_choi_est)
labels = [op.pauli_string for op in estimate.out_ops]
plot_pauli_transfer_matrix(ptm, ax, title='Estimate')

In [ ]:
fig, ax = plt.subplots()
ideal_ptm = choi2pauli_liouville(ideal)
labels = [op.pauli_string for op in estimate.out_ops]
plot_pauli_transfer_matrix(ideal_ptm, ax, title='Ideal')